# Bayesian Optimizer using Gaussian Process

In [ ]:
import time
t1 = time.time()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Input
from tensorflow.keras.layers import Reshape, MaxPooling2D
from tensorflow.keras.layers import Conv2D, Dense, Flatten
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.models import load_model

In [ ]:
# Python package scikit-optimize (or skopt) for finding the best choices of these hyper-parameters.
import skopt
from skopt import gp_minimize, forest_minimize
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_convergence
from skopt.plots import plot_objective, plot_evaluations
from skopt.plots import plot_histogram, plot_objective_2D
from skopt.utils import use_named_args

## Hyper-Parameters
1. The learning-rate

2. The number of fully-connected / dense layers.

3. The number of nodes for each of the dense layers.

4. Activation function('sigmoid' or 'relu').

5. Batch Size

6. Optimizer 


In [ ]:
# search-ranges for hyper-parqameter
batch_size = 128
range_learning_rate = Real(low=1e-6, high=1e-2, prior='log-uniform', name='learning_rate')
range_num_dense_layers = Integer(low=1, high=5, name='num_dense_layers')
range_num_dense_nodes = Integer(low=5, high=512, name='num_dense_nodes')
activation_function = Categorical(categories=['relu', 'sigmoid'], name='activation')

dimensions = [range_learning_rate, range_num_dense_layers, range_num_dense_nodes, activation_function]

In [ ]:
default_parameters = [1e-5, 1, 16, 'relu']

In [ ]:
# log the training-progress for all parameter-combinations
def log_dir_name(learning_rate, num_dense_layers, num_dense_nodes, activation):

    # The dir-name for the TensorBoard log-dir.
    s = "./logs/lr_{0:.0e}_layers_{1}_nodes_{2}_{3}/"
    
    log_dir = s.format(learning_rate, num_dense_layers, 
                       num_dense_nodes, activation)
    return log_dir

## Load MNIST Dataset

70.000 images and class-numbers for the images.
- Training-set:		48999
- Validation-set:	14000
- Test-set:		7001

In [ ]:
from sklearn.model_selection import train_test_split
DATASET_SIZE = 70000
TRAIN_RATIO = 5/7
VALIDATION_RATIO = (1-5/7)/4
TEST_RATIO = ((1-5/7)/4)*3
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape((60000,784)).astype('float32') / 255.0
y_train = tf.keras.utils.to_categorical(y_train)

x_test = x_test.reshape((10000,784)).astype('float32') / 255.0
y_test = tf.keras.utils.to_categorical(y_test)

x = np.concatenate([x_train, x_test])
y = np.concatenate([y_train, y_test])

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=(1-TRAIN_RATIO))
x_val, x_test, y_val, y_test = train_test_split(
    x_val, y_val, test_size=((TEST_RATIO/(VALIDATION_RATIO+TEST_RATIO))))

print('Training data: {}. {}'.format(x_train.shape, y_train.shape))
print('Validation data: {}. {}'.format(x_val.shape, y_val.shape))
print('Test data: {}. {}'.format(x_test.shape, y_test.shape))

In [ ]:
img_size = 28
img_size_flat = 784
img_shape = (28, 28)
img_shape_full = (28, 28, 1)
num_classes = 10
num_channels = 1
validation_data = (x_val, y_val) 

## Helper-function for plotting images

In [ ]:
def plot_images(images, cls_true, cls_pred=None):
    assert len(images) == len(cls_true) == 9
    
    # Create figure with 3x3 sub-plots.
    fig, axes = plt.subplots(3, 3)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        # Plot image.
        ax.imshow(images[i].reshape(img_shape), cmap='binary')

        # Show true and predicted classes.
        if cls_pred is None:
            xlabel = "True: {0}".format(cls_true[i])
        else:
            xlabel = "True: {0}, Pred: {1}".format(cls_true[i], cls_pred[i])

        # Show the classes as the label on the x-axis.
        ax.set_xlabel(xlabel)
        
        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])
    
    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()

In [ ]:
y_test_cls = [np.argmax(y, axis=None, out=None) for y in y_test]
images = x_test[0:9]
cls_true = y_test_cls[0:9]
plot_images(images=images, cls_true=cls_true)

In [ ]:
def plot_example_errors(cls_pred):
    # Boolean array whether the predicted class is incorrect.
    incorrect = (cls_pred != y_test_cls)

    # Get the images from the test-set that have been
    # incorrectly classified.
    images = x_test[incorrect]
    
    # Get the predicted classes for those images.
    cls_pred = cls_pred[incorrect]

    # Get the true classes for those images.
    cls_true = y_test_cls[incorrect]
    
    # Plot the first 9 images.
    plot_images(images=images[0:9],
                cls_true=cls_true[0:9],
                cls_pred=cls_pred[0:9])

## Hyper-Parameter Optimization

In [ ]:
def create_model(learning_rate, num_dense_layers, num_dense_nodes, activation):
    
    model = Sequential()
    # Add fully-connected / dense layers.
    for i in range(num_dense_layers):
        name = 'layer_dense_{0}'.format(i+1)
        model.add(Dense(num_dense_nodes, activation=activation, name=name))
    # Last fully-connected / dense layer with softmax-activation
    model.add(Dense(num_classes, activation='softmax'))
    
    # Use the Adam method for training the network.
    optimizer = Adam(learning_rate=learning_rate)
    #optimizer = sel_optimizer(learning_rate=learning_rate)
    
    # In Keras we need to compile the model so it can be trained.
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    
    return model

path_best_model = 'best_trained_model.h5'
best_accuracy = 0.0

In [ ]:
@use_named_args(dimensions=dimensions)
def fitness(learning_rate, num_dense_layers, num_dense_nodes, activation):
    # Print the hyper-parameters.
    print('learning rate: {0:.1e}'.format(learning_rate))
    print('num_dense_layers:', num_dense_layers)
    print('num_dense_nodes:', num_dense_nodes)
    print('activation:', activation)
    print()
    
    # Create the neural network with these hyper-parameters.
    model = create_model(learning_rate=learning_rate, num_dense_layers=num_dense_layers,
                         num_dense_nodes=num_dense_nodes, activation=activation)


    # Dir-name for the TensorBoard log-files.
    log_dir = log_dir_name(learning_rate, num_dense_layers, 
                           num_dense_nodes, activation)

    # saves the log-files for TensorBoard.
    callback_log = TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=True,
                               write_grads=False, write_images=False)
   
    # Use Keras to train the model.
    history = model.fit(x=x_train, y=y_train, 
                        epochs=3, batch_size=batch_size, 
                        validation_data=validation_data, 
                        callbacks=[callback_log])

    # Get the classification accuracy on the validation-set
    accuracy = history.history['val_accuracy'][-1]
    print()
    print("Accuracy: {0:.2%}".format(accuracy))
    print()

    # Save the new model
    global best_accuracy
    if accuracy > best_accuracy:
        model.save(path_best_model)
        best_accuracy = accuracy

    # Delete the Keras model with these hyper-parameters from memory.
    del model
    K.clear_session()
    return -accuracy

## Run Test for default hyper parameters

In [ ]:
fitness(x=default_parameters)

## Run the Hyper-Parameter Optimization

In [ ]:
%%time
search_result = gp_minimize(func=fitness, dimensions=dimensions, 
                            acq_func='EI', # Expected Improvement.
                            n_calls=15, x0=default_parameters)

## Optimization Progress

In [ ]:
plot_convergence(search_result)

In [ ]:
# Find the best Hyper-Parameters
search_result.x

In [ ]:
def point_to_dict(point):
    return {"lat": point.y, "lng": point.x}

In [ ]:
space = search_result.space
#space.point_to_dict(search_result.x)

In [ ]:
# This is a negative number because the Bayesian optimizer performs minimization
search_result.fun

In [ ]:
sorted(zip(search_result.func_vals, search_result.x_iters))

## Visualization of Hyper-parameter Search

In [ ]:
# activation parameter, which shows the distribution of samples during the hyper-parameter optimization.
fig = plot_histogram(result=search_result, dimension_identifier='activation')

In [ ]:
# learning_rate and num_dense_layers
fig = plot_objective_2D(result=search_result, 
                        dimension_identifier1='learning_rate', 
                        dimension_identifier2='num_dense_layers', 
                        levels=50)

In [ ]:
dim_names = ['learning_rate', 'num_dense_nodes', 'num_dense_layers']

fig = plot_objective(result=search_result, plot_dims =dim_names)

In [ ]:
fig = plot_evaluations(result=search_result, plot_dims=dim_names)

## Evaluate Best Model on TestSet

In [ ]:
model = load_model(path_best_model)
result = model.evaluate(x=x_test, y=y_test)

In [ ]:
for name, value in zip(model.metrics_names, result):
    print(name, value)

In [ ]:
print("{0}: {1:.2%}".format(model.metrics_names[1], result[1]))

In [ ]:
# Visualization of Test results
images = x_test[0:9]
cls_true = y_test_cls[0:9]
y_pred = model.predict(x=images)
cls_pred = np.argmax(y_pred,axis=1)

plot_images(images=images, cls_true=cls_true, cls_pred=cls_pred)

In [ ]:
t2 = time.time()

In [ ]:
t = t2-t1

In [ ]:
time.strftime("%H:%M:%S",time.gmtime(t))